In [ ]:
import boto3
import json
import uuid

In [ ]:
lambda_client = boto3.client("lambda")
s3 = boto3.client("s3")

In [ ]:
session = boto3.session.Session()
sts = session.client("sts")
# Get current account ID
account_id = sts.get_caller_identity()["Account"]
# Get current region
region = session.region_name

In [ ]:
S3_BUCKET = f"bedrock-mm-{account_id}-{region}"
S3_KEY_TASK_TEMPLATE = "tasks/{task_id}/upload/{file_name}"

LAMBDA_FUN_NAME_NOVA_MME_START_TASK = "bedrock-mm-nova-srv-start-task"
LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK = "bedrock-mm-nova-srv-delete-video-task"
NOVA_MME_MODEL_ID = "amazon.nova-2-multimodal-embeddings-v1:0"

## Invoke video task - Nova MME

In [ ]:
local_path = "./samples/content-moderation-demo.mp4"
file_name = local_path.split('/')[-1]

# generate task_id
task_id = str(uuid.uuid4())

# Construct target s3 key
s3_key = S3_KEY_TASK_TEMPLATE.format(task_id=task_id, file_name=file_name)

# The other values required by the request
task_name = file_name # Set your task name, default using the file name
request_by = "workshop-user" # Set your request user name

In [ ]:
s3.upload_file(local_path, S3_BUCKET, s3_key)

In [ ]:
# Inovke MME task
request = {
          "ModelId": NOVA_MME_MODEL_ID,
          "EmbedMode": "AUDIO_VIDEO_COMBINED",
          "DurationS": 5,
          "TaskId": task_id,
          "FileName": file_name,
          "TaskName": file_name,
          "Video": {
            "S3Object": {
              "Bucket": S3_BUCKET,
              "Key": s3_key
            }
          },
          "RequestBy": request_by
        }

In [ ]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_NOVA_MME_START_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps(request),
)

# Parse the Lambda response
response_payload = response["Payload"].read()

In [ ]:
response_payload

## Cleanup
We will call the delete-task endpoint, which removes all task-related state, including the uploaded video (in S3), all metadata files generated in S3, and the metadata stored in DynamoDB. For MME tasks, it will also delete the video-related vectors from the S3 vector index.

In [ ]:
response = lambda_client.invoke(
    FunctionName=LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK,
    InvocationType="RequestResponse",  # async
    Payload=json.dumps({"TaskId": task_id}),
)

# Parse the Lambda response
response_payload = response["Payload"].read()
response_payload